In [1]:
%load_ext autoreload
%autoreload 2

from collections import defaultdict
import glob
import itertools
import time

import matplotlib.pyplot as plt
import pandas as pd
from pyarrow.lib import ArrowIOError
import seaborn as sns
from tqdm import tqdm
import vaex

print(f"pd.__version__: {pd.__version__}")
print(f"vaex.__file__: {vaex.__file__}")
print(f"vaex.__version__: {vaex.__version__}")

pd.__version__: 0.25.3
vaex.__file__: /home/olga/miniconda3/envs/khtools--encodings-compare-cli--py37/lib/python3.7/site-packages/vaex/__init__.py
vaex.__version__: 1.0.0-beta.6


In [4]:
%%time
output_folder = '/mnt/data_sm/olga/kmer-hashing/visual-transduction-ensembl'
parquet = 'visual-transduction-proteins-ensembl-cleaned-with-species-metadata.parquet'

kmer_similarity_merged_filled = pd.read_parquet(f"{output_folder}/{parquet}")
print(kmer_similarity_merged_filled.shape)
kmer_similarity_merged_filled.head()

(76260328, 30)
CPU times: user 1min 18s, sys: 1min 48s, total: 3min 6s
Wall time: 41.3 s


,id2,ksize,jaccard,molecule,protein_ids__human_target_x,is_ortholog,encoding,is_ortholog_str,encoding__is_ortholog,dn_ds,...,target__perc_pos,target__protein_id,target__seq,target__species,target__taxon_id,taxonomy_level,type,protein_ids__human_target_y,target__species_cleaned,target__species__divergence_time_from_human
id1,,,,,,,,,,,,,,,,,,,,,
B0207.3b.1,ENSP00000230361,44,0.0,botvinnik,ENSP00000230361__B0207.3b.1,False,botvinnik8,(not orthologous),botvinnik8 (not orthologous),NaN,...,NaN,None,None,None,NaN,None,None,None,None,NaN
B0207.3b.1,ENSP00000360502,39,0.0,hsdm17,ENSP00000360502__B0207.3b.1,False,hsdm17,(not orthologous),hsdm17 (not orthologous),NaN,...,NaN,None,None,None,NaN,None,None,None,None,NaN
B0207.3b.1,ENSP00000360502,40,0.0,hsdm17,ENSP00000360502__B0207.3b.1,False,hsdm17,(not orthologous),hsdm17 (not orthologous),NaN,...,NaN,None,None,None,NaN,None,None,None,None,NaN
B0207.3b.1,ENSP00000360502,41,0.0,hsdm17,ENSP00000360502__B0207.3b.1,False,hsdm17,(not orthologous),hsdm17 (not orthologous),NaN,...,NaN,None,None,None,NaN,None,None,None,None,NaN
B0207.3b.1,ENSP00000360502,42,0.0,hsdm17,ENSP00000360502__B0207.3b.1,False,hsdm17,(not orthologous),hsdm17 (not orthologous),NaN,...,NaN,None,None,None,NaN,None,None,None,None,NaN


In [3]:
kmer_similarity_merged_filled.head()

,id2,ksize,jaccard,molecule,protein_ids__human_target_x,is_ortholog,encoding,is_ortholog_str,encoding__is_ortholog,dn_ds,...,target__perc_pos,target__protein_id,target__seq,target__species,target__taxon_id,taxonomy_level,type,protein_ids__human_target_y,target__species_cleaned,target__species__divergence_time_from_human
id1,,,,,,,,,,,,,,,,,,,,,
B0207.3b.1,ENSP00000230361,44,0.0,botvinnik,ENSP00000230361__B0207.3b.1,False,botvinnik8,(not orthologous),botvinnik8 (not orthologous),NaN,...,NaN,None,None,None,NaN,None,None,None,None,NaN
B0207.3b.1,ENSP00000360502,39,0.0,hsdm17,ENSP00000360502__B0207.3b.1,False,hsdm17,(not orthologous),hsdm17 (not orthologous),NaN,...,NaN,None,None,None,NaN,None,None,None,None,NaN
B0207.3b.1,ENSP00000360502,40,0.0,hsdm17,ENSP00000360502__B0207.3b.1,False,hsdm17,(not orthologous),hsdm17 (not orthologous),NaN,...,NaN,None,None,None,NaN,None,None,None,None,NaN
B0207.3b.1,ENSP00000360502,41,0.0,hsdm17,ENSP00000360502__B0207.3b.1,False,hsdm17,(not orthologous),hsdm17 (not orthologous),NaN,...,NaN,None,None,None,NaN,None,None,None,None,NaN
B0207.3b.1,ENSP00000360502,42,0.0,hsdm17,ENSP00000360502__B0207.3b.1,False,hsdm17,(not orthologous),hsdm17 (not orthologous),NaN,...,NaN,None,None,None,NaN,None,None,None,None,NaN


In [5]:
gnat1_human_ensembl = 'ENSP00000232461'

In [35]:
gnat1 = kmer_similarity_merged_filled.query('id2 == @gnat1_human_ensembl')
gnat1 = gnat1.reset_index()
print(gnat1.shape)
gnat1.head()

(2514984, 31)


,id1,id2,ksize,jaccard,molecule,protein_ids__human_target_x,is_ortholog,encoding,is_ortholog_str,encoding__is_ortholog,...,target__perc_pos,target__protein_id,target__seq,target__species,target__taxon_id,taxonomy_level,type,protein_ids__human_target_y,target__species_cleaned,target__species__divergence_time_from_human
0,B0207.3b.1,ENSP00000232461,43,0.0,hsdm17,ENSP00000232461__B0207.3b.1,True,hsdm17,(orthologous),hsdm17 (orthologous),...,47.3039,B0207.3b.1,MAFSCFDKFSYTYCYQCMHGPDGCMVPSRNGEGTELYAHSEELEAK...,caenorhabditis_elegans,6239.0,Opisthokonta,ortholog_many2many,ENSP00000232461__B0207.3b.1,Caenorhabditis elegans,797.0
1,B0207.3b.1,ENSP00000232461,43,0.0,hsdm17,ENSP00000232461__B0207.3b.1,True,hsdm17,(orthologous),hsdm17 (orthologous),...,47.3039,B0207.3b.1,MAFSCFDKFSYTYCYQCMHGPDGCMVPSRNGEGTELYAHSEELEAK...,caenorhabditis_elegans,6239.0,Opisthokonta,ortholog_many2many,ENSP00000232461__B0207.3b.1,Caenorhabditis elegans,797.0
2,B0207.3b.1,ENSP00000232461,44,0.0,hsdm17,ENSP00000232461__B0207.3b.1,True,hsdm17,(orthologous),hsdm17 (orthologous),...,47.3039,B0207.3b.1,MAFSCFDKFSYTYCYQCMHGPDGCMVPSRNGEGTELYAHSEELEAK...,caenorhabditis_elegans,6239.0,Opisthokonta,ortholog_many2many,ENSP00000232461__B0207.3b.1,Caenorhabditis elegans,797.0
3,B0207.3b.1,ENSP00000232461,44,0.0,hsdm17,ENSP00000232461__B0207.3b.1,True,hsdm17,(orthologous),hsdm17 (orthologous),...,47.3039,B0207.3b.1,MAFSCFDKFSYTYCYQCMHGPDGCMVPSRNGEGTELYAHSEELEAK...,caenorhabditis_elegans,6239.0,Opisthokonta,ortholog_many2many,ENSP00000232461__B0207.3b.1,Caenorhabditis elegans,797.0
4,B0207.3b.1,ENSP00000232461,45,0.0,hsdm17,ENSP00000232461__B0207.3b.1,True,hsdm17,(orthologous),hsdm17 (orthologous),...,47.3039,B0207.3b.1,MAFSCFDKFSYTYCYQCMHGPDGCMVPSRNGEGTELYAHSEELEAK...,caenorhabditis_elegans,6239.0,Opisthokonta,ortholog_many2many,ENSP00000232461__B0207.3b.1,Caenorhabditis elegans,797.0


In [37]:
gnat1_jaccard_greater0 = gnat1.query('jaccard > 0')
print(gnat1_jaccard_greater0.shape)
gnat1_jaccard_greater0.head()

(555226, 31)


,id1,id2,ksize,jaccard,molecule,protein_ids__human_target_x,is_ortholog,encoding,is_ortholog_str,encoding__is_ortholog,...,target__perc_pos,target__protein_id,target__seq,target__species,target__taxon_id,taxonomy_level,type,protein_ids__human_target_y,target__species_cleaned,target__species__divergence_time_from_human
31,B0207.3b.1,ENSP00000232461,3,0.266892,hsdm17,ENSP00000232461__B0207.3b.1,True,hsdm17,(orthologous),hsdm17 (orthologous),...,47.3039,B0207.3b.1,MAFSCFDKFSYTYCYQCMHGPDGCMVPSRNGEGTELYAHSEELEAK...,caenorhabditis_elegans,6239.0,Opisthokonta,ortholog_many2many,ENSP00000232461__B0207.3b.1,Caenorhabditis elegans,797.0
32,B0207.3b.1,ENSP00000232461,3,0.266892,hsdm17,ENSP00000232461__B0207.3b.1,True,hsdm17,(orthologous),hsdm17 (orthologous),...,47.3039,B0207.3b.1,MAFSCFDKFSYTYCYQCMHGPDGCMVPSRNGEGTELYAHSEELEAK...,caenorhabditis_elegans,6239.0,Opisthokonta,ortholog_many2many,ENSP00000232461__B0207.3b.1,Caenorhabditis elegans,797.0
33,B0207.3b.1,ENSP00000232461,4,0.091445,hsdm17,ENSP00000232461__B0207.3b.1,True,hsdm17,(orthologous),hsdm17 (orthologous),...,47.3039,B0207.3b.1,MAFSCFDKFSYTYCYQCMHGPDGCMVPSRNGEGTELYAHSEELEAK...,caenorhabditis_elegans,6239.0,Opisthokonta,ortholog_many2many,ENSP00000232461__B0207.3b.1,Caenorhabditis elegans,797.0
34,B0207.3b.1,ENSP00000232461,4,0.091445,hsdm17,ENSP00000232461__B0207.3b.1,True,hsdm17,(orthologous),hsdm17 (orthologous),...,47.3039,B0207.3b.1,MAFSCFDKFSYTYCYQCMHGPDGCMVPSRNGEGTELYAHSEELEAK...,caenorhabditis_elegans,6239.0,Opisthokonta,ortholog_many2many,ENSP00000232461__B0207.3b.1,Caenorhabditis elegans,797.0
35,B0207.3b.1,ENSP00000232461,5,0.052174,hsdm17,ENSP00000232461__B0207.3b.1,True,hsdm17,(orthologous),hsdm17 (orthologous),...,47.3039,B0207.3b.1,MAFSCFDKFSYTYCYQCMHGPDGCMVPSRNGEGTELYAHSEELEAK...,caenorhabditis_elegans,6239.0,Opisthokonta,ortholog_many2many,ENSP00000232461__B0207.3b.1,Caenorhabditis elegans,797.0


In [38]:
gnat1_jaccard_greater0.id1.nunique()

7068

In [39]:
gnat1_potential_orthologs = gnat1_jaccard_greater0.query('jaccard > 0 & encoding == "dayhoff6" & ksize > 10')
print(gnat1_potential_orthologs.shape)
gnat1_potential_orthologs.head()

(24686, 31)


,id1,id2,ksize,jaccard,molecule,protein_ids__human_target_x,is_ortholog,encoding,is_ortholog_str,encoding__is_ortholog,...,target__perc_pos,target__protein_id,target__seq,target__species,target__taxon_id,taxonomy_level,type,protein_ids__human_target_y,target__species_cleaned,target__species__divergence_time_from_human
456,B0207.3b.1,ENSP00000232461,11,0.011765,dayhoff,ENSP00000232461__B0207.3b.1,True,dayhoff6,(orthologous),dayhoff6 (orthologous),...,47.3039,B0207.3b.1,MAFSCFDKFSYTYCYQCMHGPDGCMVPSRNGEGTELYAHSEELEAK...,caenorhabditis_elegans,6239.0,Opisthokonta,ortholog_many2many,ENSP00000232461__B0207.3b.1,Caenorhabditis elegans,797.0
457,B0207.3b.1,ENSP00000232461,11,0.011765,dayhoff,ENSP00000232461__B0207.3b.1,True,dayhoff6,(orthologous),dayhoff6 (orthologous),...,47.3039,B0207.3b.1,MAFSCFDKFSYTYCYQCMHGPDGCMVPSRNGEGTELYAHSEELEAK...,caenorhabditis_elegans,6239.0,Opisthokonta,ortholog_many2many,ENSP00000232461__B0207.3b.1,Caenorhabditis elegans,797.0
458,B0207.3b.1,ENSP00000232461,12,0.008850,dayhoff,ENSP00000232461__B0207.3b.1,True,dayhoff6,(orthologous),dayhoff6 (orthologous),...,47.3039,B0207.3b.1,MAFSCFDKFSYTYCYQCMHGPDGCMVPSRNGEGTELYAHSEELEAK...,caenorhabditis_elegans,6239.0,Opisthokonta,ortholog_many2many,ENSP00000232461__B0207.3b.1,Caenorhabditis elegans,797.0
460,B0207.3b.1,ENSP00000232461,12,0.008850,dayhoff,ENSP00000232461__B0207.3b.1,True,dayhoff6,(orthologous),dayhoff6 (orthologous),...,47.3039,B0207.3b.1,MAFSCFDKFSYTYCYQCMHGPDGCMVPSRNGEGTELYAHSEELEAK...,caenorhabditis_elegans,6239.0,Opisthokonta,ortholog_many2many,ENSP00000232461__B0207.3b.1,Caenorhabditis elegans,797.0
461,B0207.3b.1,ENSP00000232461,13,0.005917,dayhoff,ENSP00000232461__B0207.3b.1,True,dayhoff6,(orthologous),dayhoff6 (orthologous),...,47.3039,B0207.3b.1,MAFSCFDKFSYTYCYQCMHGPDGCMVPSRNGEGTELYAHSEELEAK...,caenorhabditis_elegans,6239.0,Opisthokonta,ortholog_many2many,ENSP00000232461__B0207.3b.1,Caenorhabditis elegans,797.0


In [40]:
gnat1_potential_orthologs.id1.nunique()

567

In [41]:
gnat1_potential_orthologs.query('is_ortholog == False')

,id1,id2,ksize,jaccard,molecule,protein_ids__human_target_x,is_ortholog,encoding,is_ortholog_str,encoding__is_ortholog,...,target__perc_pos,target__protein_id,target__seq,target__species,target__taxon_id,taxonomy_level,type,protein_ids__human_target_y,target__species_cleaned,target__species__divergence_time_from_human
4816,ENSABRP00000006532,ENSP00000232461,14,0.198813,dayhoff,ENSP00000232461__ENSABRP00000006532,False,dayhoff6,(not orthologous),dayhoff6 (not orthologous),...,NaN,None,None,None,NaN,None,None,None,None,NaN
4817,ENSABRP00000006532,ENSP00000232461,13,0.215976,dayhoff,ENSP00000232461__ENSABRP00000006532,False,dayhoff6,(not orthologous),dayhoff6 (not orthologous),...,NaN,None,None,None,NaN,None,None,None,None,NaN
4818,ENSABRP00000006532,ENSP00000232461,12,0.233038,dayhoff,ENSP00000232461__ENSABRP00000006532,False,dayhoff6,(not orthologous),dayhoff6 (not orthologous),...,NaN,None,None,None,NaN,None,None,None,None,NaN
4819,ENSABRP00000006532,ENSP00000232461,11,0.252941,dayhoff,ENSP00000232461__ENSABRP00000006532,False,dayhoff6,(not orthologous),dayhoff6 (not orthologous),...,NaN,None,None,None,NaN,None,None,None,None,NaN
4831,ENSABRP00000006532,ENSP00000232461,15,0.181548,dayhoff,ENSP00000232461__ENSABRP00000006532,False,dayhoff6,(not orthologous),dayhoff6 (not orthologous),...,NaN,None,None,None,NaN,None,None,None,None,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2514535,Y95B8A.5a.1,ENSP00000232461,15,0.130952,dayhoff,ENSP00000232461__Y95B8A.5a.1,False,dayhoff6,(not orthologous),dayhoff6 (not orthologous),...,NaN,None,None,None,NaN,None,None,None,None,NaN
2514536,Y95B8A.5a.1,ENSP00000232461,14,0.148368,dayhoff,ENSP00000232461__Y95B8A.5a.1,False,dayhoff6,(not orthologous),dayhoff6 (not orthologous),...,NaN,None,None,None,NaN,None,None,None,None,NaN
2514537,Y95B8A.5a.1,ENSP00000232461,13,0.168639,dayhoff,ENSP00000232461__Y95B8A.5a.1,False,dayhoff6,(not orthologous),dayhoff6 (not orthologous),...,NaN,None,None,None,NaN,None,None,None,None,NaN
2514538,Y95B8A.5a.1,ENSP00000232461,12,0.188791,dayhoff,ENSP00000232461__Y95B8A.5a.1,False,dayhoff6,(not orthologous),dayhoff6 (not orthologous),...,NaN,None,None,None,NaN,None,None,None,None,NaN


In [10]:
# from khtools import ensembl


In [65]:
import requests, sys

def lookup(ensembl_id, expand=False):
    
    suffix = '?expand=1' if expand else ''

    server = "https://rest.ensembl.org"
    ext = f"/lookup/id/{ensembl_id}{suffix}"

    r = requests.get(server+ext, headers={ "Content-Type" : "application/json"})

    if not r.ok:
        r.raise_for_status()
        sys.exit()

    decoded = r.json()
    return decoded

def get_display_name(ensembl_protein_id):
    protein_response = lookup(ensembl_protein_id)
    transcript_id = protein_response['Parent']
    transcript_response = lookup(transcript_id)
    gene_id = transcript_response["Parent"]
    gene_response = lookup(gene_id)
    try:
        display_name = gene_response['display_name']
    except KeyError:
        display_name = gene_id
    return display_name

In [66]:
get_display_name('ENSABRP00000006532')

'GNAT2'

In [67]:
lookup('ENSABRP00000006532')

{'start': 585968,
 'id': 'ENSABRP00000006532',
 'db_type': 'core',
 'Parent': 'ENSABRT00000009395',
 'object_type': 'Translation',
 'length': 363,
 'species': 'anser_brachyrhynchus',
 'end': 588919}

In [68]:
lookup('ENSABRT00000009395')

{'strand': 1,
 'display_name': 'GNAT2-202',
 'id': 'ENSABRT00000009395',
 'version': 1,
 'assembly_name': 'ASM259213v1',
 'biotype': 'protein_coding',
 'end': 588919,
 'Parent': 'ENSABRG00000006004',
 'logic_name': 'ensembl',
 'start': 585968,
 'is_canonical': 1,
 'seq_region_name': 'NXHY01000310.1',
 'source': 'ensembl',
 'species': 'anser_brachyrhynchus',
 'object_type': 'Transcript',
 'db_type': 'core'}

In [69]:
lookup('ENSABRG00000006004')

{'seq_region_name': 'NXHY01000310.1',
 'version': 1,
 'strand': 1,
 'logic_name': 'ensembl',
 'biotype': 'protein_coding',
 'start': 585968,
 'id': 'ENSABRG00000006004',
 'species': 'anser_brachyrhynchus',
 'end': 590289,
 'assembly_name': 'ASM259213v1',
 'display_name': 'GNAT2',
 'db_type': 'core',
 'source': 'ensembl',
 'description': 'G protein subunit alpha transducin 2 [Source:HGNC Symbol;Acc:HGNC:4394]',
 'object_type': 'Gene'}

In [70]:
gnat1_potential_orthologs_ids = gnat1_potential_orthologs.id1.unique()
len(gnat1_potential_orthologs_ids)

567

In [ ]:
%%time
gnat1_potential_orthologs_ids_to_gene_names = {x: get_display_name(x) for x in tqdm(gnat1_potential_orthologs_ids)}



  0%|          | 0/567 [00:00<?, ?it/s]

  0%|          | 1/567 [00:02<27:57,  2.96s/it]

  0%|          | 2/567 [00:05<27:44,  2.95s/it]

  1%|          | 3/567 [00:08<27:32,  2.93s/it]

  1%|          | 4/567 [00:11<26:54,  2.87s/it]

  1%|          | 5/567 [00:14<26:22,  2.82s/it]

  1%|          | 6/567 [00:16<26:18,  2.81s/it]

  1%|          | 7/567 [00:19<26:09,  2.80s/it]

  1%|▏         | 8/567 [00:22<25:38,  2.75s/it]

  2%|▏         | 9/567 [00:25<25:27,  2.74s/it]

  2%|▏         | 10/567 [00:27<25:30,  2.75s/it]

  2%|▏         | 11/567 [00:30<25:26,  2.75s/it]

  2%|▏         | 12/567 [00:33<24:58,  2.70s/it]

  2%|▏         | 13/567 [00:35<24:48,  2.69s/it]

  2%|▏         | 14/567 [00:38<25:08,  2.73s/it]

  3%|▎         | 15/567 [00:41<25:21,  2.76s/it]

  3%|▎         | 16/567 [00:44<25:20,  2.76s/it]

  3%|▎         | 17/567 [00:47<25:59,  2.84s/it]

  3%|▎         | 18/567 [00:50<27:56,  3.05s/it]

  3%|▎         | 19/567 [00:54<28:12,  3.09s/it]

  4%|▎         |

In [ ]:
1+1

## Plot optimal alphabets and kmersizes against each other in pairplots